<a href="https://colab.research.google.com/github/ruanroloff/my_colab/blob/main/vaex_studies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Basic Imports
import pandas as pd
import vaex 
import sklearn
import numpy as np
import matplotlib.pyplot as plt

# ML Imports
import vaex.ml
from sklearn.linear_model import SGDRegressor
from vaex.ml.sklearn import IncrementalPredictor

# Settings
vaex.multithreading.thread_count_default = 8

# 1. I/O Kung-Fu

In [ ]:
# df = vaex.from_csv('allyears.csv', convert=True, chunk_size=1_000_000,low_memory = False)

In [ ]:
%%time
flights = vaex.open('all_years.hdf5')
flights

CPU times: user 1.31 s, sys: 184 ms, total: 1.49 s
Wall time: 4.31 s


#,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
0,1987,10,14,3,741.0,730,912.0,849,'PS',1451,'nan',91.0,79.0,nan,23.0,11.0,'SAN','SFO',447.0,nan,nan,0,'nan',0,nan,0.0,0.0,0.0,0.0,'',''
1,1987,10,15,4,729.0,730,903.0,849,'PS',1451,'nan',94.0,79.0,nan,14.0,-1.0,'SAN','SFO',447.0,nan,nan,0,'nan',0,nan,0.0,0.0,0.0,0.0,'',''
2,1987,10,17,6,741.0,730,918.0,849,'PS',1451,'nan',97.0,79.0,nan,29.0,11.0,'SAN','SFO',447.0,nan,nan,0,'nan',0,nan,0.0,0.0,0.0,0.0,'',''
3,1987,10,18,7,729.0,730,847.0,849,'PS',1451,'nan',78.0,79.0,nan,-2.0,-1.0,'SAN','SFO',447.0,nan,nan,0,'nan',0,nan,0.0,0.0,0.0,0.0,'',''
4,1987,10,19,1,749.0,730,922.0,849,'PS',1451,'nan',93.0,79.0,nan,33.0,19.0,'SAN','SFO',447.0,nan,nan,0,'nan',0,nan,0.0,0.0,0.0,0.0,'',''
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"123,534,964",2008,12,13,6,1002.0,959,1204.0,1150,'DL',1636,'N646DL',122.0,111.0,71.0,14.0,3.0,'ATL','IAD',533.0,6.0,45.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''
"123,534,965",2008,12,13,6,834.0,835,1021.0,1023,'DL',1637,'N908DL',167.0,168.0,139.0,-2.0,-1.0,'ATL','SAT',874.0,5.0,23.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''
"123,534,966",2008,12,13,6,655.0,700,856.0,856,'DL',1638,'N671DN',121.0,116.0,85.0,0.0,-5.0,'PBI','ATL',545.0,24.0,12.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''
"123,534,967",2008,12,13,6,1251.0,1240,1446.0,1437,'DL',1639,'N646DL',115.0,117.0,89.0,9.0,11.0,'IAD','ATL',533.0,13.0,13.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''


# 2. Big Data Analytics 

<img src="dask.png" width=750 height=750 />

In [ ]:
flights_filtered = flights[((flights.ActualElapsedTime>=0).fillmissing(True)) &
                 ((flights.CRSElapsedTime>=0).fillmissing(True)) &
                 ((flights.AirTime>0).fillmissing(True)) & 
                 ((flights.Distance > 0).fillmissing(True))]

In [ ]:
flights_filtered

#,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,CRSElapsedTime,AirTime,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,IsArrDelayed,IsDepDelayed
0,1995,1,6,5,657.0,645,952.0,937,'UA',482,'N7298U',115.0,112.0,83.0,15.0,12.0,'ORD','PHL',678.0,7.0,25.0,0,'nan',0,0.0,0.0,0.0,0.0,0.0,'',''
1,1995,1,7,6,648.0,645,938.0,937,'UA',482,'N7449U',110.0,112.0,88.0,1.0,3.0,'ORD','PHL',678.0,5.0,17.0,0,'nan',0,0.0,0.0,0.0,0.0,0.0,'',''
2,1995,1,8,7,649.0,645,932.0,937,'UA',482,'N7453U',103.0,112.0,83.0,-5.0,4.0,'ORD','PHL',678.0,3.0,17.0,0,'nan',0,0.0,0.0,0.0,0.0,0.0,'',''
3,1995,1,9,1,645.0,645,928.0,937,'UA',482,'N7288U',103.0,112.0,84.0,-9.0,0.0,'ORD','PHL',678.0,3.0,16.0,0,'nan',0,0.0,0.0,0.0,0.0,0.0,'',''
4,1995,1,10,2,645.0,645,931.0,937,'UA',482,'N7275U',106.0,112.0,82.0,-6.0,0.0,'ORD','PHL',678.0,6.0,18.0,0,'nan',0,0.0,0.0,0.0,0.0,0.0,'',''
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"84,186,963",2008,12,13,6,1002.0,959,1204.0,1150,'DL',1636,'N646DL',122.0,111.0,71.0,14.0,3.0,'ATL','IAD',533.0,6.0,45.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''
"84,186,964",2008,12,13,6,834.0,835,1021.0,1023,'DL',1637,'N908DL',167.0,168.0,139.0,-2.0,-1.0,'ATL','SAT',874.0,5.0,23.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''
"84,186,965",2008,12,13,6,655.0,700,856.0,856,'DL',1638,'N671DN',121.0,116.0,85.0,0.0,-5.0,'PBI','ATL',545.0,24.0,12.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''
"84,186,966",2008,12,13,6,1251.0,1240,1446.0,1437,'DL',1639,'N646DL',115.0,117.0,89.0,9.0,11.0,'IAD','ATL',533.0,13.0,13.0,0,--,0,0.0,0.0,0.0,0.0,0.0,'',''


In [ ]:
%%time
flights_filtered[flights_filtered.Distance > 4500].groupby(['Origin', 'Dest', 'Distance'], 
                                                 agg={'Origin':'count'})

CPU times: user 2.64 s, sys: 826 ms, total: 3.47 s
Wall time: 26.2 s


#,Origin,Dest,Distance,count
0,'ATL','HNL',4502,5187
1,'HNL','ATL',4502,5302
2,'HNL','EWR',4962,3808
3,'EWR','HNL',4962,3801


# 3. Machine Learning @ Scale & Locally 

In [ ]:
df_train, df_test = flights.ml.train_test_split(test_size=0.2, verbose=False)
print(f"Train Set has {round(df_train.shape[0]/1e6,0)} million rows.")

Train Set has 99.0 million rows.


In [ ]:
features = ["Month","DayofMonth","DayOfWeek","TailNum","Origin","Dest","DepDelay"]
target = "DepDelay"
cat_features = ["Month","DayofMonth","DayOfWeek","Origin","Dest"]

In [ ]:
df_train = df_train[features]
df_test = df_test[features]

In [ ]:
df_train

#,Month,DayofMonth,DayOfWeek,TailNum,Origin,Dest,DepDelay
0,7,19,7,'nan','FAT','RNO',1.0
1,7,20,1,'nan','FAT','RNO',35.0
2,7,21,2,'nan','FAT','RNO',0.0
3,7,22,3,'nan','FAT','RNO',-1.0
4,7,23,4,'nan','FAT','RNO',42.0
...,...,...,...,...,...,...,...
"98,827,970",12,13,6,'N646DL','ATL','IAD',3.0
"98,827,971",12,13,6,'N908DL','ATL','SAT',-1.0
"98,827,972",12,13,6,'N671DN','PBI','ATL',-5.0
"98,827,973",12,13,6,'N646DL','IAD','ATL',11.0


In [ ]:
%%time
df_train.describe()

CPU times: user 21.3 s, sys: 6.08 s, total: 27.4 s
Wall time: 2min 56s


,Month,DayofMonth,DayOfWeek,TailNum,Origin,Dest,DepDelay
data_type,int64,int64,int64,string,string,string,float64
count,98827975,98827975,98827975,98442555,98827975,98827975,96788650
NA,0,0,0,385420,0,0,2039325
mean,6.5784710149125285,15.72339076056147,3.94076323024933,--,--,--,8.520332683635942
std,3.43204,8.787011,1.990207,--,--,--,29.911976
min,1,1,1,--,--,--,-1410.0
max,12,31,7,--,--,--,2601.0


In [ ]:
df_train = df_train.dropna()
df_test = df_test.dropna()

In [ ]:
df_train.shape,df_test.shape

((96544931, 7), (24444183, 7))

## 3.1 Feature Transformations

In [ ]:
%%time
bayes_encoder = vaex.ml.BayesianTargetEncoder(features= cat_features, 
                                              target=target)

df_train = bayes_encoder.fit_transform(df_train)
df_train

CPU times: user 1min 28s, sys: 9.22 s, total: 1min 37s
Wall time: 5min 17s


#,Month,DayofMonth,DayOfWeek,TailNum,Origin,Dest,DepDelay,mean_encoded_Month,mean_encoded_DayofMonth,mean_encoded_DayOfWeek,mean_encoded_Origin,mean_encoded_Dest
0,7,19,7,'nan','FAT','RNO',1.0,10.418567462224562,9.414129876534204,8.947884812292255,5.755802763392751,9.573642125251586
1,7,20,1,'nan','FAT','RNO',35.0,10.418567462224562,8.88287956797837,8.37606377810181,5.755802763392751,9.573642125251586
2,7,21,2,'nan','FAT','RNO',0.0,10.418567462224562,9.374941856681339,7.011983756390205,5.755802763392751,9.573642125251586
3,7,22,3,'nan','FAT','RNO',-1.0,10.418567462224562,9.863194128863165,7.863812029589218,5.755802763392751,9.573642125251586
4,7,23,4,'nan','FAT','RNO',42.0,10.418567462224562,9.088171353789297,9.589669989236226,5.755802763392751,9.573642125251586
...,...,...,...,...,...,...,...,...,...,...,...,...
"96,544,926",12,13,6,'N646DL','ATL','IAD',3.0,11.723357643157794,8.314252183104161,7.097430572403026,11.160642143186545,8.579877845626042
"96,544,927",12,13,6,'N908DL','ATL','SAT',-1.0,11.723357643157794,8.314252183104161,7.097430572403026,11.160642143186545,8.830743702300287
"96,544,928",12,13,6,'N671DN','PBI','ATL',-5.0,11.723357643157794,8.314252183104161,7.097430572403026,8.01297852142236,9.177049212340295
"96,544,929",12,13,6,'N646DL','IAD','ATL',11.0,11.723357643157794,8.314252183104161,7.097430572403026,10.085413187395309,9.177049212340295


In [ ]:
final_features = (df_train.get_column_names(regex="mean_encoded*"))
all_cols = final_features + [target]
all_cols,final_features

(['mean_encoded_Month',
  'mean_encoded_DayofMonth',
  'mean_encoded_DayOfWeek',
  'mean_encoded_Origin',
  'mean_encoded_Dest',
  'DepDelay'],
 ['mean_encoded_Month',
  'mean_encoded_DayofMonth',
  'mean_encoded_DayOfWeek',
  'mean_encoded_Origin',
  'mean_encoded_Dest'])

In [ ]:
df_train = df_train[all_cols]
df_train

#,mean_encoded_Month,mean_encoded_DayofMonth,mean_encoded_DayOfWeek,mean_encoded_Origin,mean_encoded_Dest,DepDelay
0,10.418567462224562,9.414129876534204,8.947884812292255,5.755802763392751,9.573642125251586,1.0
1,10.418567462224562,8.88287956797837,8.37606377810181,5.755802763392751,9.573642125251586,35.0
2,10.418567462224562,9.374941856681339,7.011983756390205,5.755802763392751,9.573642125251586,0.0
3,10.418567462224562,9.863194128863165,7.863812029589218,5.755802763392751,9.573642125251586,-1.0
4,10.418567462224562,9.088171353789297,9.589669989236226,5.755802763392751,9.573642125251586,42.0
...,...,...,...,...,...,...
"96,544,926",11.723357643157794,8.314252183104161,7.097430572403026,11.160642143186545,8.579877845626042,3.0
"96,544,927",11.723357643157794,8.314252183104161,7.097430572403026,11.160642143186545,8.830743702300287,-1.0
"96,544,928",11.723357643157794,8.314252183104161,7.097430572403026,8.01297852142236,9.177049212340295,-5.0
"96,544,929",11.723357643157794,8.314252183104161,7.097430572403026,10.085413187395309,9.177049212340295,11.0


## 3.2 Online Learning Model

In [ ]:
# Base Model
model= SGDRegressor(learning_rate = "constant",eta0=1e-4)

# Vaex Model Wrapper
vaex_model = IncrementalPredictor(features=final_features,
                                  target=target,
                                  model=model,
                                  num_epochs=5,
                                  shuffle=False,
                                  prediction_name="Predicted_Departure_Delay"
                                 )

vaex_model.fit(df_train,progress="widget")

In [ ]:
df_train = vaex_model.transform(df_train)
df_train.head(100)["DepDelay","Predicted_Departure_Delay"]

#,DepDelay,Predicted_Departure_Delay
0,1.0,25.89117967455455
1,35.0,21.618379562174795
2,0.0,28.99297588939099
3,-1.0,32.33119879415561
4,42.0,21.478434617754047
...,...,...
95,3.0,16.36341966282299
96,-5.0,17.766796900147156
97,1.0,7.75663321302855
98,15.0,5.631430924439776


## 3.3 Automatic Pipelines Magic

In [ ]:
pipeline = df_train.state_get()

In [ ]:
pipeline

{'virtual_columns': {'mean_encoded_Month': '_map(__Month, map_key_set, map_choices, use_missing=True, axis=None)',
  'mean_encoded_DayOfWeek': '_map(__DayOfWeek, map_key_set_2, map_choices_2, use_missing=True, axis=None)',
  'mean_encoded_Dest': '_map(__Dest, map_key_set_4, map_choices_4, use_missing=True, axis=None)',
  'mean_encoded_Origin': '_map(__Origin, map_key_set_3, map_choices_3, use_missing=True, axis=None)',
  'mean_encoded_DayofMonth': '_map(__DayofMonth, map_key_set_1, map_choices_1, use_missing=True, axis=None)',
  '__Predicted_Departure_Delay': 'incremental_prediction_function(mean_encoded_Month, mean_encoded_DayofMonth, mean_encoded_DayOfWeek, mean_encoded_Origin, mean_encoded_Dest)',
  'Predicted_Departure_Delay': 'incremental_prediction_function_1(mean_encoded_Month, mean_encoded_DayofMonth, mean_encoded_DayOfWeek, mean_encoded_Origin, mean_encoded_Dest)'},
 'column_names': ['mean_encoded_Month',
  'mean_encoded_DayofMonth',
  'mean_encoded_DayOfWeek',
  'mean_encoded

In [ ]:
df_test.state_set(pipeline)

## 3.4 Fast Inference

In [ ]:
df_test.sample(20)[final_features + ["Predicted_Departure_Delay"]]

#,mean_encoded_Month,mean_encoded_DayofMonth,mean_encoded_DayOfWeek,mean_encoded_Origin,mean_encoded_Dest,Predicted_Departure_Delay
0,11.723357643157794,7.659928270681211,8.947884812292255,6.376443004395522,6.453997953190668,3.1835729040238903
1,11.042133233689276,8.022578287606114,8.947884812292255,9.366233139352836,8.788104818361726,12.260974592954753
2,7.091558584105692,7.706367617458895,7.863812029589218,7.7427760354673225,8.579877845626042,16.575734047643223
3,11.723357643157794,9.169806452526979,7.011983756390205,2.973728130904325,8.750665912526282,22.174484220168154
4,5.308326000560051,9.155795172832487,7.097430572403026,6.861085735734224,8.929106474094231,35.30724117196442
...,...,...,...,...,...,...
15,6.986778982915546,8.070849213954824,8.37606377810181,11.643050790718577,6.408816843669631,19.480728929926656
16,11.723357643157794,7.659928270681211,7.011983756390205,6.886401959905261,6.745930741324233,7.295940453418822
17,5.308326000560051,8.107813821801342,10.616276913973326,11.643050790718577,11.869361688417849,24.518303142072952
18,10.418567462224562,9.169806452526979,7.863812029589218,8.719192000653726,8.77067702960511,26.324892416058674
